<a href="https://colab.research.google.com/github/greyhound101/licenseplate/blob/main/scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
np.set_printoptions(suppress=True)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os

In [ ]:
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

In [ ]:
classes = ['license_plate']

In [ ]:
def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [ ]:
classes = ['license_plate']
def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

#     dw = np.float32(1. / int(size[0]))
#     dh = np.float32(1. / int(size[1]))
    dw=1
    dh=1
    
    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]
def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = '/kaggle/working/Dataset/labels/' +  img_jpg_file_name + '.npy'
    #file_path = open(save_file_name, "a+")
    total=[]
    for box in img_box:
            cls_num = classes.index(box[0])
            total.append([cls_num,int(box[1]),int(box[2]),int(box[3]),int(box[4])])
    np.save(save_file_name,total)
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        if cls_name=='license_plate':
            x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
            y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
            x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
            y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
            area=(y2-y1)*(x2-x1)
            if (area>10) & (area<20000):
                img_jpg_file_name = img_xml_file + '.jpg'
                img_box.append([cls_name, x1, x2,y1 , y2])

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_txt_file(img_xml_file, [img_w, img_h], img_box)
    return len(img_box)

In [ ]:
files = os.listdir('/kaggle/input/number-plates/VOC2007/Annotations')
for file in files:
    file_xml = file.split(".")
    get_xml_data('/kaggle/input/number-plates/VOC2007/Annotations', file_xml[0])

In [ ]:
import glob
total=[]
for path in glob.glob('/kaggle/working/Dataset/labels/car*.npy'):
    a=np.load(path)
    id=path.split('/')[-1].split('.')[0].split('_')[-1]
    for i in a:
        total.append([id,0,i[1],i[2],i[3],i[4]])
import pandas as pd
df=pd.DataFrame(total)
df.columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
df['x']=df['XMin']
df['y']=df['YMin']
df['w']=df['XMax']-df['XMin']
df['h']=df['YMax']-df['YMin']
df['ImageID']=df['ImageID'].astype('int64')

In [ ]:
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

In [ ]:
import glob
total=[]
sum=0
for path in glob.glob('/kaggle/input/single-pred/yolov5/*.npy'):
    a=np.load(path,allow_pickle=True)
    if len(a)==0:
        sum+=1
    id=path.split('/')[-1].split('.')[0].split('_')[0]
    for i in a:
            total.append([int(id),0,float(i['scores'].split(' ')[-1]),i['boxes'][0],i['boxes'][1],i['boxes'][2],i['boxes'][3]])


In [ ]:
org=pd.DataFrame(total)
org.columns=['ImageID', 'LabelName','Conf', 'XMin', 'YMin', 'XMax', 'YMax']
org['x']=org['XMin']
org['y']=org['YMin']
org['w']=org['XMax']-org['XMin']
org['h']=org['YMax']-org['YMin']
org['ImageID']=org['ImageID'].astype('int64')
pre=org.sort_values(by='Conf',ascending=False)
tot=0

In [ ]:
for en,i in enumerate(list(set(pre['ImageID'].values).intersection(set(df['ImageID'].values)))):
    tot+=calculate_precision(df.loc[df['ImageID']==i][['x','y','w','h']].values.copy(), pre.loc[pre['ImageID']==i][['x','y','w','h']].values, threshold=0.5, form='coco')
tot/en

In [ ]:
import glob
total=[]
sum=0
for path in glob.glob('/kaggle/input/nms-wbf/yolov5/*.npy'):
    a=np.load(path,allow_pickle=True)
    if len(a)==0:
        sum+=1
    id=path.split('/')[-1].split('.')[0].split('_')[0]
    for i in a:
            total.append([int(id),0,float(i[-1]),i[0],i[1],i[2],i[3]])


In [ ]:
org=pd.DataFrame(total)
org.columns=['ImageID', 'LabelName','Conf', 'XMin', 'YMin', 'XMax', 'YMax']
org['x']=org['XMin']
org['y']=org['YMin']
org['w']=org['XMax']-org['XMin']
org['h']=org['YMax']-org['YMin']
org['ImageID']=org['ImageID'].astype('int64')
pre=org.sort_values(by='Conf',ascending=False)
tot=0

In [ ]:
for en,i in enumerate(list(set(pre['ImageID'].values).intersection(set(df['ImageID'].values)))):
    tot+=calculate_precision(df.loc[df['ImageID']==i][['x','y','w','h']].values.copy(), pre.loc[pre['ImageID']==i][['x','y','w','h']].values, threshold=0.5, form='coco')
tot/en